In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import geopandas as gpd #A more flexible package to work with geospatial data in python 
import networkx as nx # networkx package 
import osmnx as ox 
import pandas as pd #Base package for data analysis and manipulation 
import random
import shapely.geometry #python package for basic spatial operation 

In [ ]:
g_namen = ['Amsterdam', 'Den_Haag', 'Rotterdam', 'Utrecht']#set the city names
project_gemeenten = gpd.read_file(r"data\boundaries\UrbanRunner_Areas.geojson") #download city geometries from file
project_gemeenten["extent"] = project_gemeenten.envelope.buffer(2000).envelope.to_crs("EPSG:4326") #set additional geometry column of extent
project_gemeenten.set_index('gemeentenaam', inplace=True) #set index to gemeentenaam column
project_gemeenten.to_crs("EPSG:4326", inplace=True) #project to WGS84

##### Preparing network
The code section below reads the edge and nodes with the costs and attributes from file. Before transforming the edge and node dataframes to a Networkx graph, it is possible to make changes to the edges or their attributes, as well as to the final cost columns. This allows for flexibility that is often needed when doing analysis.

In [ ]:
# get network
for g in g_namen:
    globals()[f"{g}_nodes"] = gpd.read_file(f'data/graphs/with_costs/{g}_graph_costs.gpkg', driver = "GPKG", layer= 'nodes').set_index('osmid') 
    globals()[f"{g}_edges"] = gpd.read_file(f'data/graphs/with_costs/{g}_graph_costs.gpkg', driver = "GPKG", layer= 'edges').set_index(['u', 'v', 'key'])
    edges = globals()[f"{g}_edges"]

    #---ANY CHANGES TO EDGES SHOULD BE MADE HERE---#
    nodes_disruptions = globals()[f"{g}_nodes"]['barrier']
    if g == 'Utrecht':
        edges = edges.join(nodes_disruptions.rename_axis('u'), how = 'left', rsuffix = "_obstnodes_u")
    edges['cost_UHI'] = edges['UHI_mean'] / 3
    edges['cost_UHI'][edges['UHI_mean'] < 0] = 0  
    edges['cost_greenspace'][(edges['landuse'].isnull()) & (edges['leisure'].isnull()) & (edges['natural'].isnull())] = 1
    edges['cost_disruptions'] = 0
    edges['cost_disruptions'][(
        edges['footway'] == 'crossing') |
        (edges['cycleway'] == 'crossing') |
        (edges['barrier_obstnodes_u'] == 'gate')|
        (edges['barrier_obstnodes_u'] == 'wicket_gate')|
        (edges['barrier_obstnodes_u'] == 'stile')|
        (edges['barrier_obstnodes_u'] == 'turnstile')|
        (edges['barrier_obstnodes_u'] == 'full-height_turnstile')] = 0.5
    edges['cost_disruptions'][(edges['highway_obstnodes_u'] == 'traffic_signals')] = 1
    edges['index_waterpoints1'][(edges['index_waterpoints1'].isnull())] = 0.1 #a value of 0.1 is given to allow searching for the number of waterpoints. Else an error was given
    #---ANY CHANGES TO EDGES SHOULD BE MADE HERE---#
    
    #final cost column are added before analysis to gain more flexibility in changing the cost function
    edges['final_cost_willemijn'] = ((0.7*edges['cost_UHI']) + (0.9*edges['cost_greenspace']) + (0.5*edges['cost_grade']) + (0.1*edges['cost_disruptions']) + (0.3*edges['cost_surface_prefpaved'])) * edges['length']
    edges['final_cost_rachid'] = ((0.5*edges['cost_waterpoints']) + (0.7*edges['cost_UHI']) + (0.9*edges['cost_greenspace']) + (0.3*edges['cost_disruptions']) + (0.1*edges['cost_surface_prefunpaved'])) * edges['length']
    globals()[f"{g}_edges"] = edges
    globals()[f"{g}_network_both"] = ox.graph_from_gdfs(globals()[f"{g}_nodes"],globals()[f"{g}_edges"])

In [ ]:

def get_random_XY_in_polygon(poly): 
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p): # if statement checks if the random X, Y pair is within the 
            return p.coords[0][0], p.coords[0][1]

In [ ]:
#getting random nodes in the network
n_routes = 250
for g in g_namen:
    X_list = []
    Y_list = []    
    for i in range((n_routes*2)):
        x, y = get_random_XY_in_polygon(project_gemeenten.to_crs('epsg:28992').loc[g].geometry)
        X_list.append(x)
        Y_list.append(y)
    random_nodes = ox.nearest_nodes(globals()[f"{g}_network_both"], X_list, Y_list)
    source_nodes = random_nodes[0:n_routes]
    goal_nodes = random_nodes[n_routes:(n_routes*2)]

In [ ]:
errors = 0
SameNodesCount = 0
for g in g_namen:    
    for routenr in range(n_routes):
        print(routenr)
        source = source_nodes[routenr]
        goal = goal_nodes[routenr]
        if source == goal:
            print('yeah, what is the chance to randomly select the same node?')
            SameNodesCount += 1
            continue
        # calculate routes ## !! Error handling needs to be included in case no route can be found!
        try:
            WillemijnRoute = nx.dijkstra_path(G=globals()[f"{g}_network_both"], source=source, target=goal, weight='final_cost_willemijn')
            RachidRoute = nx.dijkstra_path(G=globals()[f"{g}_network_both"], source=source, target=goal, weight='final_cost_rachid')
            shortestroute = nx.dijkstra_path(G=globals()[f"{g}_network_both"], source=source, target=goal, weight='length')
        
        
            # extract edge attributes of the routes
            #CRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Gs, coolestroute)) 
            #SRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Gs, shortestroute)) 
            WRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(globals()[f"{g}_network_both"], WillemijnRoute))
            RRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(globals()[f"{g}_network_both"], RachidRoute))
            SRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(globals()[f"{g}_network_both"], shortestroute))
            if routenr == 0:
                RouteStats = pd.DataFrame([[source, goal, 
                                    sum(WRstats.UHI_mean*WRstats.length)/sum(WRstats.length),                             
                                    sum(RRstats.UHI_mean*RRstats.length)/sum(RRstats.length),                                   
                                    sum(SRstats.UHI_mean*SRstats.length)/sum(SRstats.length),

                                    sum(WRstats.length - (WRstats.cost_greenspace*WRstats.length))/sum(WRstats.length),
                                    sum(RRstats.length - (RRstats.cost_greenspace*RRstats.length))/sum(RRstats.length),
                                    sum(SRstats.length - (SRstats.cost_greenspace*SRstats.length))/sum(SRstats.length),

                                    sum(WRstats.grade*WRstats.length)/sum(WRstats.length),
                                    sum(SRstats.grade*SRstats.length)/sum(SRstats.length),

                                    sum(WRstats.cost_surface_prefpaved*WRstats.length)/sum(WRstats.length),
                                    sum(RRstats.cost_surface_prefunpaved*RRstats.length)/sum(RRstats.length),
                                    sum(SRstats.cost_surface_prefpaved*SRstats.length)/sum(SRstats.length),
                                    sum(SRstats.cost_surface_prefunpaved*SRstats.length)/sum(SRstats.length),

                                    len(RRstats[RRstats.cost_waterpoints == 0]),
                                    len(SRstats[SRstats.cost_waterpoints == 0]),
                                    len(RRstats['index_waterpoints1'].unique()) -1,
                                    len(SRstats['index_waterpoints1'].unique()) -1,

                                    len(WRstats[WRstats.cost_disruptions > 0]),
                                    len(RRstats[RRstats.cost_disruptions > 0]),
                                    len(SRstats[SRstats.cost_disruptions > 0]),

                                    len(WRstats[WRstats.cost_disruptions == 1])/2,
                                    len(RRstats[RRstats.cost_disruptions == 1])/2,
                                    len(SRstats[SRstats.cost_disruptions == 1])/2,

                                    
                                    sum(WRstats.length), sum(RRstats.length), sum(SRstats.length)]],
                                columns=['source','target',
                                    'WR_UHI','RR_UHI','SR_UHI',
                                    'WR_greenspace', 'RR_greenspace', 'SR_greenspace', 
                                    'WR_grade', 'SR_grade',
                                    'WR_prefpaved','RR_prefunpaved','SR_prefpaved','SR_prefunpaved', 
                                    'RR_water','SR_water', 'RR_waterpoints', 'SR_waterpoints',
                                    'WR_disruption','RR_disruption','SR_disruption',
                                    'WR_trafficsignals','RR_trafficsignals','SR_trafficsignals', 
                                    'WR_length', 'RR_length', 'SR_length'])
            else:
                RouteStats = RouteStats.append(pd.DataFrame([[source, goal, 
                                    sum(WRstats.UHI_mean*WRstats.length)/sum(WRstats.length),                             
                                    sum(RRstats.UHI_mean*RRstats.length)/sum(RRstats.length),                                   
                                    sum(SRstats.UHI_mean*SRstats.length)/sum(SRstats.length),

                                    sum(WRstats.length - (WRstats.cost_greenspace*WRstats.length))/sum(WRstats.length),
                                    sum(RRstats.length - (RRstats.cost_greenspace*RRstats.length))/sum(RRstats.length),
                                    sum(SRstats.length - (SRstats.cost_greenspace*SRstats.length))/sum(SRstats.length),

                                    sum(WRstats.grade*WRstats.length)/sum(WRstats.length),
                                    sum(SRstats.grade*SRstats.length)/sum(SRstats.length),

                                    sum(WRstats.cost_surface_prefpaved*WRstats.length)/sum(WRstats.length),
                                    sum(RRstats.cost_surface_prefunpaved*RRstats.length)/sum(RRstats.length),
                                    sum(SRstats.cost_surface_prefpaved*SRstats.length)/sum(SRstats.length),
                                    sum(SRstats.cost_surface_prefunpaved*SRstats.length)/sum(SRstats.length),

                                    len(RRstats[RRstats.cost_waterpoints == 0]),
                                    len(SRstats[SRstats.cost_waterpoints == 0]),
                                    len(RRstats['index_waterpoints1'].unique()) -1,
                                    len(SRstats['index_waterpoints1'].unique()) -1,

                                    len(WRstats[WRstats.cost_disruptions > 0]),
                                    len(RRstats[RRstats.cost_disruptions > 0]),
                                    len(SRstats[SRstats.cost_disruptions > 0]),

                                    len(WRstats[WRstats.cost_disruptions == 1])/2,
                                    len(RRstats[RRstats.cost_disruptions == 1])/2,
                                    len(SRstats[SRstats.cost_disruptions == 1])/2,

                                    
                                    sum(WRstats.length), sum(RRstats.length), sum(SRstats.length)]],
                                columns=['source','target',
                                    'WR_UHI','RR_UHI','SR_UHI',
                                    'WR_greenspace', 'RR_greenspace', 'SR_greenspace', 
                                    'WR_grade', 'SR_grade',
                                    'WR_prefpaved','RR_prefunpaved','SR_prefpaved','SR_prefunpaved', 
                                    'RR_water','SR_water', 'RR_waterpoints', 'SR_waterpoints',
                                    'WR_disruption','RR_disruption','SR_disruption',
                                    'WR_trafficsignals','RR_trafficsignals','SR_trafficsignals', 
                                    'WR_length', 'RR_length', 'SR_length']),
                        ignore_index=True)
        except Exception as ex:
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print(message)
            errors += 1

In [ ]:
#saving the stats to excel
RouteStats.to_excel('routestats_rotterdam_250_2.xlsx')